In [104]:
import tensorflow as tf
gpus = tf.config.list_physical_devices("GPU")
if gpus:
    for gpu in gpus:
        print("Found a GPU with the name:", gpu)
else:
    print("Failed to detect a GPU.")

Found a GPU with the name: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [126]:
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU, Bidirectional, Dense, Embedding, Input

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
import pandas as pd
from termcolor import colored
from collections import Counter
from nltk.corpus import stopwords
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
#import nltk
#nltk.download('popular')


In [106]:
df=pd.read_csv(r'D:\Deep Learning Internship_Codexcue\Special Task 1_RealvsFakeNews\news_articles.csv\news_articles.csv')
df.head()

,author,published,title,text,language,site_url,main_img_url,type,label,title_without_stopwords,text_without_stopwords,hasImage
0,Barracuda Brigade,2016-10-26T21:41:00.000+03:00,muslims busted they stole millions in govt ben...,print they should pay all the back all the mon...,english,100percentfedup.com,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,bias,Real,muslims busted stole millions govt benefits,print pay back money plus interest entire fami...,1.0
1,reasoning with facts,2016-10-29T08:47:11.259+03:00,re why did attorney general loretta lynch plea...,why did attorney general loretta lynch plead t...,english,100percentfedup.com,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,bias,Real,attorney general loretta lynch plead fifth,attorney general loretta lynch plead fifth bar...,1.0
2,Barracuda Brigade,2016-10-31T01:41:49.479+02:00,breaking weiner cooperating with fbi on hillar...,red state \nfox news sunday reported this mor...,english,100percentfedup.com,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,bias,Real,breaking weiner cooperating fbi hillary email ...,red state fox news sunday reported morning ant...,1.0
3,Fed Up,2016-11-01T05:22:00.000+02:00,pin drop speech by father of daughter kidnappe...,email kayla mueller was a prisoner and torture...,english,100percentfedup.com,http://100percentfedup.com/wp-content/uploads/...,bias,Real,pin drop speech father daughter kidnapped kill...,email kayla mueller prisoner tortured isis cha...,1.0
4,Fed Up,2016-11-01T21:56:00.000+02:00,fantastic trumps point plan to reform healthc...,email healthcare reform to make america great ...,english,100percentfedup.com,http://100percentfedup.com/wp-content/uploads/...,bias,Real,fantastic trumps point plan reform healthcare ...,email healthcare reform make america great sin...,1.0


In [107]:
df = df.dropna()
#df = df.interpolate(method='linear')

In [108]:
df['language']

0       english
1       english
2       english
3       english
4       english
         ...   
2041    english
2042    english
2043    english
2044    english
2045    english
Name: language, Length: 2045, dtype: object

In [109]:
df=df.drop(['title','text','site_url','main_img_url'], axis=1)


In [110]:
print(df['language'].value_counts())
print(df['label'].value_counts())
print(df['type'].value_counts())
print(df['hasImage'].value_counts())

language
english    1967
german       72
ignore        3
french        2
spanish       1
Name: count, dtype: int64
label
Fake    1291
Real     754
Name: count, dtype: int64
type
bs            598
conspiracy    430
bias          389
hate          244
satire        146
state         121
junksci       102
fake           15
Name: count, dtype: int64
hasImage
1.0    1580
0.0     465
Name: count, dtype: int64


In [111]:

# Preprocessing function
from bs4 import BeautifulSoup
import string

def preprocess_text(text):
    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation
    text = ''.join([char for char in text if char not in string.punctuation])
    # Tokenize the text
    tokens = word_tokenize(text)
    # Lemmatize tokens
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    # Join tokens back into a single string
    preprocessed_text = ' '.join(lemmatized_tokens)
    return preprocessed_text


# Apply preprocessing to the 'review' column
df['title_without_stopwords'] = df['title_without_stopwords'].apply(preprocess_text)
df['text_without_stopwords'] = df['text_without_stopwords'].apply(preprocess_text)
print ('preprocessed data')
# Display the preprocessed DataFrame
print(df.head(3))

preprocessed data
                 author                      published language  type label  \
0     Barracuda Brigade  2016-10-26T21:41:00.000+03:00  english  bias  Real   
1  reasoning with facts  2016-10-29T08:47:11.259+03:00  english  bias  Real   
2     Barracuda Brigade  2016-10-31T01:41:49.479+02:00  english  bias  Real   

                             title_without_stopwords  \
0           muslim busted stole million govt benefit   
1         attorney general loretta lynch plead fifth   
2  breaking weiner cooperating fbi hillary email ...   

                              text_without_stopwords  hasImage  
0  print pay back money plus interest entire fami...       1.0  
1  attorney general loretta lynch plead fifth bar...       1.0  
2  red state fox news sunday reported morning ant...       1.0  


In [85]:
df.replace({'label': {'Real':1.0}}, inplace=True)
df.replace({'label': {'Fake':0.0}}, inplace=True)
#df['label'].replace('Real', 1.0, inplace=True)
#df['label'].replace('Fake', 0.0, inplace=True)
Y=df['label']
df=df.drop('label',axis=1)
X=df['text_without_stopwords']
print(X.head())
print(Y.value_counts())

0    print pay back money plus interest entire fami...
1    attorney general loretta lynch plead fifth bar...
2    red state fox news sunday reported morning ant...
3    email kayla mueller prisoner tortured isi chan...
4    email healthcare reform make america great sin...
Name: text_without_stopwords, dtype: object
label
0.0    1291
1.0     754
Name: count, dtype: int64


C:\Users\HF\AppData\Local\Temp\ipykernel_8472\186706190.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({'label': {'Fake':0.0}}, inplace=True)


In [86]:
#vocab_size=5000
tokenizer = Tokenizer( )
#tokenizer = Tokenizer(num_words=vocab_size )
tokenizer.fit_on_texts(X)
w_idx=tokenizer.word_index
vocab_size=len(w_idx)
print(vocab_size)

42238


In [88]:
tokenizer.document_count
sequences = tokenizer.texts_to_sequences(X)


2045

In [90]:
len(sequences)

2045

In [91]:
from keras.utils import pad_sequences
# Get the minimum and the maximum length of reviews
print("Max length of a review:: ", len(max((sequences), key=len)))
print("Min length of a review:: ", len(min((sequences), key=len)))
# Keeping a fixed length of all reviews to max words
#max_words = len(max((sequences), key=len))
#sequences = pad_sequences(sequences,padding='post',maxlen=max_words)
max_length=150
sequences = pad_sequences(sequences,maxlen=max_length)
sequences.shape

Max length of a review::  3015
Min length of a review::  1


(2045, 150)

In [92]:
sequences[0]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,   946,   408,    55,   127,
        1565,   233,   493,   125,   370,   277,    59,  4612, 11771,
          42,    44,    14,  6809,    47,    80,    63,  3993,    33,
        1966,    63,

In [93]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,SimpleRNN,Embedding,Flatten,Dropout,GlobalMaxPooling1D

In [94]:
X_train, X_test, y_train, y_test = train_test_split(sequences, Y, test_size=0.3, random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1431, 150)
(1431,)
(614, 150)
(614,)


In [99]:
embd_len=150
model=Sequential()
model.add(Embedding(input_dim=vocab_size+1,output_dim=embd_len,input_length=max_length))
#model.add(Dropout(0.5))
model.add(LSTM(embd_len))
#model.add(Dropout(rate = 0.5))
#model.add(Dense(units = 32,  activation = 'relu'))
#model.add(Dropout(0.5))
#model.add(GlobalMaxPooling1D( ))
#model.add(Dense(64,activation='relu'))
#model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_12 (Embedding)    (None, 150, 150)          6335850   
                                                                 
 lstm_12 (LSTM)              (None, 150)               180600    
                                                                 
 dense_11 (Dense)            (None, 1)                 151       
                                                                 
Total params: 6,516,601
Trainable params: 6,516,601
Non-trainable params: 0
_________________________________________________________________


In [100]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

model.fit(X_train,y_train,batch_size=32,epochs=10,validation_data=(X_test,y_test))
# Printing model score on test data
print()
print("RNN Score on test dataset---> ", model.evaluate(X_test, y_test, verbose=0))

Epoch 1/10
45/45 [==============================] - 3s 39ms/step - loss: 0.6424 - accuracy: 0.6303 - val_loss: 0.5593 - val_accuracy: 0.7182
Epoch 2/10
45/45 [==============================] - 1s 30ms/step - loss: 0.3185 - accuracy: 0.8798 - val_loss: 0.6756 - val_accuracy: 0.7085
Epoch 3/10
45/45 [==============================] - 1s 30ms/step - loss: 0.0685 - accuracy: 0.9818 - val_loss: 0.8860 - val_accuracy: 0.7003
Epoch 4/10
45/45 [==============================] - 1s 31ms/step - loss: 0.0683 - accuracy: 0.9853 - val_loss: 0.7332 - val_accuracy: 0.7101
Epoch 5/10
45/45 [==============================] - 1s 31ms/step - loss: 0.0295 - accuracy: 0.9944 - val_loss: 1.0314 - val_accuracy: 0.7003
Epoch 6/10
45/45 [==============================] - 1s 31ms/step - loss: 0.0232 - accuracy: 0.9937 - val_loss: 0.8781 - val_accuracy: 0.7052
Epoch 7/10
45/45 [==============================] - 1s 30ms/step - loss: 0.0372 - accuracy: 0.9895 - val_loss: 0.7331 - val_accuracy: 0.7296
Epoch 8/10
45

In [112]:
News_type = df[['type','language']]

X2 = News_type.values

y2 = df['label']

In [113]:
print (X2.shape)
X2

(2045, 2)


In [121]:
from sklearn import preprocessing

# label_encoder object knows how to understand word labels.
feature_encoder1 = preprocessing.OneHotEncoder()

# Encode labels in column 'species'.
XX2 = feature_encoder1.fit_transform(X2)
label_encoder2 = preprocessing.LabelEncoder()

# Encode labels in column 'species'.
yy2 = label_encoder2.fit_transform(y2)
X_train, X_test, y_train, y_test = train_test_split(XX2, yy2, test_size=0.20, random_state=42)

#from keras.utils import to_categorical
#y_train = to_categorical(y_train)
#y_test = to_categorical(y_test)
y_train

In [129]:
input2 = Input(shape=(13,))
dense_layer_1 = Dense(32, activation='relu')(input2)
dense_layer_2 = Dense(32, activation='relu')(dense_layer_1)
output = Dense(1, activation='sigmoid')(dense_layer_2)

model = Model(inputs=input2, outputs=output)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [130]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 13)]              0         
                                                                 
 dense_18 (Dense)            (None, 32)                448       
                                                                 
 dense_19 (Dense)            (None, 32)                1056      
                                                                 
 dense_20 (Dense)            (None, 1)                 33        
                                                                 
Total params: 1,537
Trainable params: 1,537
Non-trainable params: 0
_________________________________________________________________


In [131]:
model = Model(inputs=input2, outputs=output)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [133]:
history = model.fit(X_train, y_train, batch_size=16, epochs=10, verbose=1, validation_data=(X_test,y_test))

Epoch 1/10


C:\ProgramData\anaconda3\envs\my_env\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_2/dense_18/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model_2/dense_18/embedding_lookup_sparse/Reshape:0", shape=(None, 32), dtype=float32), dense_shape=Tensor("gradient_tape/model_2/dense_18/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


103/103 [==============================] - 1s 7ms/step - loss: 0.0000e+00 - acc: 0.6167 - val_loss: 0.0000e+00 - val_acc: 0.6455
Epoch 2/10
103/103 [==============================] - 1s 5ms/step - loss: 0.0000e+00 - acc: 0.6278 - val_loss: 0.0000e+00 - val_acc: 0.6455
Epoch 3/10
103/103 [==============================] - 1s 5ms/step - loss: 0.0000e+00 - acc: 0.6278 - val_loss: 0.0000e+00 - val_acc: 0.6455
Epoch 4/10
103/103 [==============================] - 1s 5ms/step - loss: 0.0000e+00 - acc: 0.6278 - val_loss: 0.0000e+00 - val_acc: 0.6455
Epoch 5/10
103/103 [==============================] - 1s 6ms/step - loss: 0.0000e+00 - acc: 0.6278 - val_loss: 0.0000e+00 - val_acc: 0.6455
Epoch 6/10
103/103 [==============================] - 1s 8ms/step - loss: 0.0000e+00 - acc: 0.6278 - val_loss: 0.0000e+00 - val_acc: 0.6455
Epoch 7/10
103/103 [==============================] - 1s 7ms/step - loss: 0.0000e+00 - acc: 0.6278 - val_loss: 0.0000e+00 - val_acc: 0.6455
Epoch 8/10
103/103 [===========

In [134]:
score = model.evaluate(X_test, y_test, verbose=1)

print("Test Score:", score[0])
print("Test Accuracy:", score[1])

13/13 [==============================] - 0s 6ms/step - loss: 0.0000e+00 - acc: 0.6455
Test Score: 0.0
Test Accuracy: 0.645476758480072


In [135]:
News_type = df[['type','language']]

X2 = News_type.values

y2 = df['label']
News_type

In [145]:
#vocab_size=5000
tokenizer = Tokenizer( )
#tokenizer = Tokenizer(num_words=vocab_size )
X2 = df['type'] + ' ' + df['language']

tokenizer.fit_on_texts(X2)
w_idx=tokenizer.word_index
vocab_size=len(w_idx)
print(vocab_size)

13


In [146]:
w_idx

{'english': 1,
 'bs': 2,
 'conspiracy': 3,
 'bias': 4,
 'hate': 5,
 'satire': 6,
 'state': 7,
 'junksci': 8,
 'german': 9,
 'fake': 10,
 'ignore': 11,
 'french': 12,
 'spanish': 13}

In [155]:
sequences1 = tokenizer.texts_to_sequences(X2)
sequences1 = pad_sequences(sequences1)
sequences1.shape
sequences1

(2045, 2)

In [156]:
X_train, X_test, y_train, y_test = train_test_split(sequences1, yy2, test_size=0.20, random_state=42)


In [159]:
input2 = Input(shape=(2,))
dense_layer_1 = Dense(10, activation='relu')(input2)
dense_layer_2 = Dense(10, activation='relu')(dense_layer_1)
output = Dense(1, activation='sigmoid')(dense_layer_2)

model = Model(inputs=input2, outputs=output)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
model.summary()

model = Model(inputs=input2, outputs=output)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

history = model.fit(X_train, y_train, batch_size=16, epochs=10, verbose=1, validation_data=(X_test,y_test))
score = model.evaluate(X_test, y_test, verbose=1)

print("Test Score:", score[0])
print("Test Accuracy:", score[1])

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 2)]               0         
                                                                 
 dense_33 (Dense)            (None, 10)                30        
                                                                 
 dense_34 (Dense)            (None, 10)                110       
                                                                 
 dense_35 (Dense)            (None, 1)                 11        
                                                                 
Total params: 151
Trainable params: 151
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
103/103 [==============================] - 1s 5ms/step - loss: 0.0000e+00 - acc: 0.6271 - val_loss: 0.0000e+00 - val_acc: 0.6455
Epoch 2/10
103/103 [==============================] 